## Завдання 1.Застосування алгоритму максимального потоку для логістики товарів

In [10]:

# Програмна реалізація Завдання 1 «Застосування алгоритму максимального потоку для логістики товарів»

import networkx as nx
from prettytable import PrettyTable

# Побудова графа
G = nx.DiGraph()

# Додаємо ребра з пропускною здатністю (термінали до складів)
G.add_edge('T1', 'S1', capacity=25)
G.add_edge('T1', 'S2', capacity=20)
G.add_edge('T1', 'S3', capacity=15)
G.add_edge('T2', 'S3', capacity=15)
G.add_edge('T2', 'S4', capacity=30)
G.add_edge('T2', 'S2', capacity=10)

# Склади до магазинів
G.add_edge('S1', 'M1', capacity=15)
G.add_edge('S1', 'M2', capacity=10)
G.add_edge('S1', 'M3', capacity=20)
G.add_edge('S2', 'M4', capacity=15)
G.add_edge('S2', 'M5', capacity=10)
G.add_edge('S2', 'M6', capacity=25)
G.add_edge('S3', 'M7', capacity=20)
G.add_edge('S3', 'M8', capacity=15)
G.add_edge('S3', 'M9', capacity=10)
G.add_edge('S4', 'M10', capacity=20)
G.add_edge('S4', 'M11', capacity=10)
G.add_edge('S4', 'M12', capacity=15)
G.add_edge('S4', 'M13', capacity=5)
G.add_edge('S4', 'M14', capacity=10)

# Додаємо джерело та стік
G.add_edge('SOURCE', 'T1', capacity=1000)
G.add_edge('SOURCE', 'T2', capacity=1000)

# З’єднуємо магазини до стоку
for i in range(1, 15):
    G.add_edge(f'M{i}', 'SINK', capacity=1000)

# Застосування алгоритму Едмондса-Карпа
flow_value, flow_dict = nx.maximum_flow(G, 'SOURCE', 'SINK')

print(f"\nМаксимальний потік у мережі: {flow_value}\n")

# Побудова очищеної таблиці: тільки потоки Термінал → Магазин
table = PrettyTable()
table.field_names = ["Термінал", "Магазин", "Фактичний Потік (одиниць)"]

for terminal in ['T1', 'T2']:
    for warehouse in flow_dict[terminal]:
        if flow_dict[terminal][warehouse] > 0:
            # Перевіряємо, чи є склад у словнику потоку
            if warehouse in flow_dict:
                for shop in flow_dict[warehouse]:
                    flow = flow_dict[warehouse][shop]
                    if flow > 0 and shop.startswith("M"):
                        table.add_row([terminal, shop, flow])

print("Потоки між терміналами та магазинами:")
print(table)


Максимальний потік у мережі: 115

Потоки між терміналами та магазинами:
+----------+---------+---------------------------+
| Термінал | Магазин | Фактичний Потік (одиниць) |
+----------+---------+---------------------------+
|    T1    |    M1   |             15            |
|    T1    |    M2   |             10            |
|    T1    |    M4   |             15            |
|    T1    |    M5   |             10            |
|    T1    |    M6   |             5             |
|    T1    |    M7   |             20            |
|    T1    |    M8   |             10            |
|    T2    |    M7   |             20            |
|    T2    |    M8   |             10            |
|    T2    |   M10   |             20            |
|    T2    |   M11   |             10            |
|    T2    |    M4   |             15            |
|    T2    |    M5   |             10            |
|    T2    |    M6   |             5             |
+----------+---------+---------------------------+


##Завдання 2. Порівняння ефективності OOBTree і словника для діапазонних запитів

In [15]:
pip install BTrees

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 5.7 MB/s eta 0:00:00


In [22]:
# Програмна реалізація Завдання 2 «Порівняння ефективності OOBTree і словника для діапазонних запитів»

from BTrees.OOBTree import OOBTree
import csv
import timeit
from typing import Dict, List
from collections import defaultdict

# Читання файлу CSV
def load_items_from_csv(file_path: str) -> list:
    items = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            items.append({
                "id": row["ID"],
                "name": row["Name"],
                "category": row["Category"],
                "price": float(row["Price"])
            })
    return items

# Функція додавання до OOBTree: ключ — ціна
def add_item_to_tree(tree: OOBTree, item: Dict):
    price = item["price"]
    if price not in tree:
        tree[price] = []
    tree[price].append(item)

# Функція діапазонний запит у OOBTree
def range_query_tree(tree: OOBTree, min_price: float, max_price: float):
    result = []
    for _, items in tree.items(min_price, max_price):
        result.extend(items)
    return result

# Функція додавання до звичайного словника
def add_item_to_dict(store: Dict, item: Dict):
    store[item["id"]] = item

# Функція діапазонний запит у словнику
def range_query_dict(store: Dict, min_price: float, max_price: float):
    return [item for item in store.values() if min_price <= item["price"] <= max_price]

# Підготовка
items = load_items_from_csv('generated_items_data.csv')
tree = OOBTree()
dict_store = {}

for item in items:
    add_item_to_tree(tree, item)
    add_item_to_dict(dict_store, item)

# Межі діапазону
min_price = 100
max_price = 200

# Вимірювання часу
print("Виконання діапазонних запитів по OOBTree...")
tree_time = timeit.timeit(lambda: range_query_tree(tree, min_price, max_price), number=100)
print(f"Total range_query time for OOBTree: {tree_time:.6f} seconds")

print("Виконання діапазонних запитів по Dict...")
dict_time = timeit.timeit(lambda: range_query_dict(dict_store, min_price, max_price), number=100)
print(f"Total range_query time for Dict: {dict_time:.6f} seconds")

Виконання діапазонних запитів по OOBTree...
Total range_query time for OOBTree: 0.000601 seconds
Виконання діапазонних запитів по Dict...
Total range_query time for Dict: 0.002204 seconds
